In [ ]:
!pip install imutils
!pip install opencv-python

In [ ]:
# import library keras
import keras

# import library open computer vision
import cv2

# import library komputasi numerik
import numpy as np

# import ilbrary pendukung open cv untuk keperluan manajemen data
from imutils import paths

# import metode dari library keras untuk load model yang sudah dilatih
from keras.models import load_model

# import library pickle untuk load label yang telah disimpan tadi
import pickle


In [ ]:
# load modelnya dulu
model = load_model('model_cnn.h5')

# load label yang disimpan dalam format pickle tadi
label = open('lable.pickle', 'rb').read()
lb = pickle.loads(label)

# cek apakah berhasil 
print(model.summary())

In [ ]:
# import library plotting
import matplotlib.pyplot as plt

# lihat sebuah gambar
file_gbr = 'datasets/animals/cats/cats_00028.jpg'

# baca gambarnya
gbr = plt.imread(file_gbr)

# kasi judul gambarnya dengan menampilkan nama_file gambarnya + dimensi gambarnya
plt.title('gambar = '+str(file_gbr.split('/')[3])+' ; ukuran = '+str(gbr.shape)+'\n')

# tampilkan
plt.imshow(gbr)

In [ ]:
# lihat sebuah gambar

# untuk jaga-jaga, simpan / backup data gambar ke sebuah variabel
output = gbr.copy()
img = gbr.copy()

# ubah dimensi / ukurannya menjadi 32 x 32
img_cnn = cv2.resize(img, (32,32))

# tambahkan dimensi gambarnnya
img_cnn = np.expand_dims(img_cnn, axis = 0)

# melakukan prediksi
pred = model.predict(img_cnn)

# proses kembali sehingga didapatkan label kelas keluaran hasil prediksinya
i = pred.argmax(axis = 1)[0]
label_class = lb.classes_[i]

# tulis label dan akurasinya di gambarnya dengan bantuan open cv putText
text = "{}: {:.2f}%".format(label_class, pred[0][i] * 100)
cv2.putText(output, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# tampilkan gambar
plt.figure(figsize=[5,5])
plt.imshow(output)

<h2> BONUS - OBJECT DETECTION [ MobileNet SSD ] </h2>

In [ ]:
import numpy as np
import argparse
import cv2

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(255, 100, size=(len(CLASSES), 3))
print("[INFO] loading model...")
prototxt = 'object_detection/MobileNetSSD_deploy.prototxt.txt'
model_proto = 'object_detection/MobileNetSSD_deploy.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model_proto)


image = cv2.imread(file_gbr)
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

print("[INFO] computing object detections...")
net.setInput(blob)
detections = net.forward()

for i in np.arange(0, detections.shape[2]):

    confidence = detections[0, 0, i, 2]

    if confidence > 0.2:

        idx = int(detections[0, 0, i, 1])
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
        print("[INFO] {}".format(label))
        cv2.rectangle(image, (startX, startY), (endX, endY),COLORS[idx], 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

plt.figure(figsize=[5,5])
plt.imshow(image)